## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/



## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made,it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insigths. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named 'Starbucks' within the radius supplied to the Foursquare API's explore' endpoint.

An area will be a circular area with a given radius. I have not determined that radius yet, and I might use more than one radius in this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use. I have done some exploratory work with a radius of 300m.

The areas to be chosen should contain a mix of areas with Starbuck's and those without. I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. I may or may not use that resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [1]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Libraries imported!


In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=750, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=2000, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
        print(point_name,':',lat,'-',lng)
         

            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [4]:
sf_lat=37.7749
sf_lon=-122.4194
ch_lat=41.8781
ch_lon=-87.6298
la_lat=34.0522
la_lon=-118.2437
starbucks_m_lat=37.744644
starbucks_m_lon=-122.452731
import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342
random_lat0 = random.uniform(37.767547,37.804203)
random_lon0 = random.uniform(-122.459264, -122.409342)
random_lat1 = random.uniform(llat,ulat)
random_lon1 = random.uniform(llon, ulon)
random_lat2 = random.uniform(llat,ulat)
random_lon2 = random.uniform(llon, ulon)
random_lat3 = random.uniform(llat,ulat)
random_lon3 = random.uniform(llon, ulon)
random_lat4 = random.uniform(llat,ulat)
random_lon4 = random.uniform(llon, ulon)
random_lat5 = random.uniform(llat,ulat)
random_lon5 = random.uniform(llon, ulon)
random_lat6 = random.uniform(llat,ulat)
random_lon6 = random.uniform(llon, ulon)
random_lat7 = random.uniform(llat,ulat)
random_lon7 = random.uniform(llon, ulon)
random_lat8 = random.uniform(llat,ulat)
random_lon8 = random.uniform(llon, ulon)
random_lat9 = random.uniform(llat,ulat)
random_lon9 = random.uniform(llon, ulon)

data = pd.DataFrame(np.array([[sf_lat, sf_lon], [ch_lat, ch_lon], [la_lat, la_lon], [starbucks_m_lat,starbucks_m_lon], [random_lat0,random_lon0], [random_lat1,random_lon1], [random_lat2,random_lon2], [random_lat3,random_lon3], [random_lat4,random_lon4], [random_lat5,random_lon5], [random_lat6,random_lon6], [random_lat7,random_lon7], [random_lat8,random_lon8], [random_lat9,random_lon9] ]), columns=['Latitude', 'Longitude'])

data



,Latitude,Longitude
0,37.774900,-122.419400
1,41.878100,-87.629800
2,34.052200,-118.243700
3,37.744644,-122.452731
4,37.775483,-122.440225
5,37.797456,-122.452972
6,37.801094,-122.451562
7,37.792571,-122.434854
8,37.803708,-122.436942
9,37.773670,-122.452635


In [5]:
random_latlons = []
for j in range(0,250):
    random_lat = random.uniform(llat,ulat)
    random_lon = random.uniform(llon,ulon)
    random_latlon=(random_lat,random_lon)
    random_latlons.append(random_latlon)
    

In [6]:
random_latlons = pd.DataFrame(random_latlons)
random_lats=random_latlons[0]
random_lons=random_latlons[1]
random_lats.shape



(250,)

In [7]:
random_latlons[0].shape

(250,)

In [8]:
random_latlons[1].shape

(250,)

In [9]:


data["Latitude"] = random_latlons[0]
data["Longitude"] = random_latlons[1]

data["Latitude"].head()
data["Latitude"].shape


(14,)

In [10]:
data["Longitude"].head()
data.shape

(14, 2)

In [11]:

#venues = getNearbyVenues(latitudes = data["Latitude"],
#                               longitudes = data["Longitude"]
#                               )
venues = getNearbyVenues(latitudes = random_lats,
                               longitudes = random_lons
                               )


point0 : 37.7923791495663 - -122.45205111382896
point1 : 37.77249133335368 - -122.41691911938867
point2 : 37.80242461710856 - -122.44804606180787
point3 : 37.77095949035393 - -122.44573434440436
point4 : 37.79680291098128 - -122.45459650688491
point5 : 37.80086807077609 - -122.45315129724125
point6 : 37.78965655601253 - -122.4444616901623
point7 : 37.77050280623158 - -122.42716118760927
point8 : 37.774025949178586 - -122.44998180884049
point9 : 37.76786824213697 - -122.4361009215237
point10 : 37.79127349949814 - -122.4590451595717
point11 : 37.79634241873818 - -122.45744882417031
point12 : 37.79348637752565 - -122.42384587446983
point13 : 37.80404857569562 - -122.41278302028064
point14 : 37.788183392195094 - -122.45467998192512
point15 : 37.7910999348201 - -122.44276214590712
point16 : 37.77519075343581 - -122.41157233237752
point17 : 37.77834442002197 - -122.42822019007335
point18 : 37.77584306566565 - -122.4508959041265
point19 : 37.787379196655245 - -122.44933629255878
point20 : 37.

In [12]:
venues.tail()

,Point Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
6705,point248,37.780128,-122.458360,Jin's Healing Massage,37.781303,-122.457254,163,Massage Studio
6706,point248,37.780128,-122.458360,Sprint Store,37.781116,-122.455795,251,Mobile Phone Shop
6707,point248,37.780128,-122.458360,Liang Kuan Alter Therapies Reflexology,37.781535,-122.456092,253,Massage Studio
6708,point248,37.780128,-122.458360,Burger King,37.781415,-122.461282,294,Fast Food Restaurant
6709,point249,37.803569,-122.437328,Kevbra Cohabitation,37.805304,-122.439232,255,Speakeasy


In [13]:
venues['Venue'].unique()
venues['Venue'].value_counts()

Starbucks                                                    25
Peet's Coffee & Tea                                          21
Chase Bank                                                   19
Pet Food Express                                             14
Whole Foods Market                                           13
Project Juice                                                13
Mel's Drive-In                                               13
Wells Fargo                                                  13
Marine Layer                                                 12
The Italian Homemade Company                                 12
Souvla                                                       12
Tennessee Watershed                                          11
Paper Source                                                 11
Boba Guys                                                    11
Garibaldi's                                                  10
The Mindful Body                        

In [14]:
venues[350:400]


# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

0    6685
1      25
Name: isStarbucks, dtype: int64

In [15]:
venues.head()

,Point Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,point0,37.792379,-122.452051,Julius Kahn Playground & Clubhouse,37.791163,-122.453511,186,Playground
1,point0,37.792379,-122.452051,Wood Line by Andy Goldsworthy,37.793462,-122.448950,298,Sculpture Garden
2,point0,37.792379,-122.452051,Julius Kahn Field,37.791097,-122.453334,181,Baseball Field
3,point0,37.792379,-122.452051,Paul Goode Field,37.793678,-122.453674,203,Baseball Field
4,point0,37.792379,-122.452051,Julius Kahn Park,37.791005,-122.454069,234,Park


In [16]:
grouped = onehot.groupby('point_name').mean().reset_index()
grouped

,point_name,point_lat,point_lon,isStarbucks,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,point0,37.792379,-122.452051,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,point1,37.772491,-122.416919,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.04,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,point10,37.791273,-122.459045,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,point100,37.771885,-122.456482,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,point101,37.785690,-122.451712,0.050000,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,point102,37.804097,-122.410299,0.000000,0.041667,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,point103,37.780815,-122.457883,0.000000,0.028571,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.028571,0.00,0.000000,0.000000,0.000000
7,point104,37.783498,-122.455910,0.000000,0.045455,0.045455,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.045455,0.045455,0.00,0.000000,0.000000,0.000000
8,point105,37.791911,-122.441351,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,point106,37.768450,-122.434922,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [ ]:
num_top_venues = 50

for point in grouped['point_name']:
    print("----"+point+"----")
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----point0----
                            venue  freq
0                  Baseball Field   0.4
1                            Park   0.2
2                Sculpture Garden   0.2
3                      Playground   0.2
4                    Intersection   0.0
5                       Newsstand   0.0
6                     Opera House   0.0
7                          Office   0.0
8         North Indian Restaurant   0.0
9                    Noodle House   0.0
10                      Nightclub   0.0
11        New American Restaurant   0.0
12           Other Great Outdoors   0.0
13                   Neighborhood   0.0
14                Nature Preserve   0.0
15                  National Park   0.0
16                     Nail Salon   0.0
17                    Music Venue   0.0
18                    Music Store   0.0
19                   Music School   0.0
20                   Optical Shop   0.0
21           Pakistani Restaurant   0.0
22              Outdoor Sculpture   0.0
23                  Movie

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X .drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
print(X.columns)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

#RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
RF = RandomForestClassifier(n_estimators=3, max_depth=30, random_state=0)  
RF.fit(X, y)  
RF.predict(X)  
RF.score(X,y)

In [ ]:
estimator = RF.estimators_[2]

In [ ]:
estimator

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = 'has_starbucks',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')